# Quick example of implementing a simple payments network

There are 4 steps to running Veriphi guardrails on a tool given to an autonomous Agent.
 - Define network classes
 - Prepare data
 - Distribute and encrypt
 - Reconstruct and recover 


In [ ]:
import { randomBytes } from 'crypto'
import * as ic from './index'
import { decryptAESCTR, encryptAESCTR } from './utils'

## Define network classes

In [ ]:
// Run a network with 4 classes
const networkClasses: Record<string,number> = {
    'Agent': 0,
    'Authoriser': 1,
    'Endpoint': 2,
}



## Prepare data

In [ ]:
type Wallet = {
  walletId: string;
  owner: {
    name: string;
    id: string;
  };
  defaultCurrency: string;
  addresses: Record<string, string>; // currency symbol -> address
};

// Assign to a variable
const wallet: Wallet = {
  walletId: "abc123",
  owner: {
    name: "Jane Doe",
    id: "user-6789"
  },
  defaultCurrency: "USD",
  addresses: {
    BTC: "1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa",
    ETH: "0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7"
  }
};

// Serialize to JSON string
const walletJson: string = JSON.stringify(wallet);
const encoder = new TextEncoder();
const walletBytes = encoder.encode(walletJson);

In [ ]:
walletBytes

Uint8Array(198) [
  123,  34, 119,  97, 108, 108, 101, 116,  73, 100,  34,  58,
   34,  97,  98,  99,  49,  50,  51,  34,  44,  34, 111, 119,
  110, 101, 114,  34,  58, 123,  34, 110,  97, 109, 101,  34,
   58,  34,  74,  97, 110, 101,  32,  68, 111, 101,  34,  44,
   34, 105, 100,  34,  58,  34, 117, 115, 101, 114,  45,  54,
   55,  56,  57,  34, 125,  44,  34, 100, 101, 102,  97, 117,
  108, 116,  67, 117, 114, 114, 101, 110,  99, 121,  34,  58,
   34,  85,  83,  68,  34,  44,  34,  97, 100, 100, 114, 101,
  115, 115, 101, 115,
  ... 98 more items
]


In [ ]:
// Generate a public key afor distribution and a private key for obfuscation
const setupNode = new ic.SetupNode('node_setup') 
const seed = Buffer.from(randomBytes(32))
const publicKey = setupNode.genPublicKey(seed)
const privateKey = setupNode.genPrivateKey('obfPrivateKey', seed)
let [encrypted,nonce] = setupNode.encryptData(Buffer.from(walletBytes), privateKey);
// optionally set conditions with a minimum spend of 0 and a maximum spend of 1000
const [lowVal, highVal] = setupNode.implementConditions(0, 1000, privateKey);
let chunkSize;
[encrypted,chunkSize] = setupNode.obfuscateData(encrypted, privateKey, lowVal,highVal,50); // Dummy test value of 50 that should pass

[
  <Buffer dc a8 c1 9c e0 9b ad 98 86 f6 e3 f8 61 d7 9a 92 21 45 ae b0 70 59 bf 0b 34 cd 1f 91 7c 14 4d 64 2d e8 26 a2 aa ed 1e ac 9f 87 60 c2 6d 79 93 a3 67 fc ... 148 more bytes>,
  6
]


In [ ]:
nonce

<Buffer 94 33 6a ce 58 3f 1e 4c 2e be eb 86 60 d0 01 b3>


In [ ]:
console.log("Chunk size is ", chunkSize)

Chunk size is  6


### The cell above can be wrapped up in 1 function call

In [ ]:
const [publicData, privateData] = ic.setupNode(walletBytes, 0, 1000, true);

In [ ]:
// Pack the data for distribution, with a specified identity for each class
let agentPacket = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Agent"])
let authPacket  = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Authoriser"])
let endpointPacket = setupNode.packageData(encrypted, publicKey, 'E', networkClasses["Endpoint"])

### Or more generally for many parties

In [ ]:

let [agentPacket, authPacket, endpointPacket] = ic.distributeData(publicData, 'E', 3);

## Distribute and encrypt

### Example code for the Agent members

In [ ]:
const agentNode = new ic.EncryptNode('agent_member')
const [publicKey, packetData, mode, identity ] = agentNode.unpackageData(agentPacket);
const agentPrivateKey = agentNode.genPrivateKey('privateKey', randomBytes(32));
const agentEncrypted = agentNode.encryptData(Buffer.from(packetData), agentPrivateKey, publicKey, mode, identity);
const agentEncPacket = agentNode.packageData(agentEncrypted, mode, identity);


### Or in simple function calls

In [ ]:
const agentEncPacket    = ic.encryptNode(agentPacket, 'agent_member');
const authEncPacket     = ic.encryptNode(authPacket, 'authoriser_member');
const endpointEncPacket = ic.encryptNode(endpointPacket, 'endpoint_member');

## Reconstruction and recovery is conditioned on complete visibility and correct conditions

In [ ]:
const veriphier = new ic.DecryptNode('Veriphier');
const partyData = veriphier.collectPackets(agentEncPacket, authEncPacket, endpointEncPacket);
const partyDataRecov = veriphier.recoverPackets(partyData);
const streamList = veriphier.reconstructData(partyDataRecov);
const reconstructed = veriphier.reassembleData(streamList, partyData[0].mode);
// attempt recovery with a spend of 500, which should pass checks
const [recovered,chunkSize] = veriphier.obfuscateData(reconstructed, privateData.key, privateData.low_val, privateData.high_val, 500);
const decrypted = veriphier.decryptData(recovered, privateData.nonce, privateData.key);


### Again, the above logic is wrapped up in a sigle call

In [ ]:
const decrypted = ic.decryptNode(privateData, 500, true, agentEncPacket, authEncPacket, endpointEncPacket);

## The original wallet is recovered

In [ ]:
const decoder = new TextDecoder
const recoveredWallet = decoder.decode(decrypted)

In [ ]:
// original wallet
console.log("Original wallet is  ", wallet)

Original wallet is   {
  walletId: 'abc123',
  owner: { name: 'Jane Doe', id: 'user-6789' },
  defaultCurrency: 'USD',
  addresses: {
    BTC: '1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa',
    ETH: '0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7'
  }
}


In [ ]:
// recovered wallet
console.log("Recovered wallet is ", JSON.parse(recoveredWallet))


Recovered wallet is  {
  walletId: 'abc123',
  owner: { name: 'Jane Doe', id: 'user-6789' },
  defaultCurrency: 'USD',
  addresses: {
    BTC: '1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa',
    ETH: '0x89205A3A3b2A69De6Dbf7f01ED13B2108B2c43e7'
  }
}


# Interfere with the scheme (change private keys, arguments, public key between parties) to break the decryption

### For example, decrypting with a spend larger then 1000 (the minimum spend)

In [ ]:
const decrypted = ic.decryptNode(privateData, 1500, true, agentEncPacket, authEncPacket, endpointEncPacket);
const decoder = new TextDecoder
const recoveredWallet = decoder.decode(decrypted)
console.log("Recovered wallet is ", JSON.parse(recoveredWallet))


<anonymous_script>:1
�Ij�<	6&�����Bd���Wo�H�e�
^

SyntaxError: Unexpected token '�', "�Ij�<	6&"... is not valid JSON
    at JSON.parse (<anonymous>)
    at evalmachine.<anonymous>:9:63
    at evalmachine.<anonymous>:11:3
    at sigintHandlersWrap (node:vm:280:12)
    at Script.runInThisContext (node:vm:135:14)
    at Object.runInThisContext (node:vm:317:38)
    at Object.execute (/Users/petervincent/Documents/veriphi-node/node_modules/tslab/dist/executor.js:160:38)
    at JupyterHandlerImpl.handleExecuteImpl (/Users/petervincent/Documents/veriphi-node/node_modules/tslab/dist/jupyter.js:250:38)
    at /Users/petervincent/Documents/veriphi-node/node_modules/tslab/dist/jupyter.js:208:57
    at async JupyterHandlerImpl.handleExecute (/Users/petervincent/Documents/veriphi-node/node_modules/tslab/dist/jupyter.js:208:21)
